In [18]:
from dotenv import load_dotenv
import os
import pandas as pd
import requests

load_dotenv()
av_api_key = os.getenv("ALPHA_VANTAGE_API_KEY")

#### Function to structure the data

In [19]:
def structure_data(data):
    """
    Converts raw Alpha Vantage crypto daily data into a sorted closing price series.
    """
    df = pd.DataFrame.from_dict(data["Time Series (Digital Currency Daily)"], orient='index')
    df.index = pd.to_datetime(df.index)
    df = df.astype(float).sort_index()
    return df["4. close"]

#### Function to fetch multiple tickers data


In [20]:
tickers = ["ETH", "BTC","USDT","XRP"]

In [21]:
def fetch_data(tickers):
    """
    Fetch closing prices for multiple tickers and combine them into one DataFrame.
    Each column represents one ticker.
    """
    market = "USD"
    combined_df = pd.DataFrame()

    for ticker in tickers:
        url = f"https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={ticker}&market={market}&apikey={av_api_key}"
        r = requests.get(url)
        data = r.json()

        # Extract and structure the series
        closing_price_series = structure_data(data)
        closing_price_series.name = ticker  # Set the column name

        # Combine using outer join to align by date
        combined_df = pd.concat([combined_df, closing_price_series], axis=1)

    return combined_df

# Example usage
df = fetch_data(tickers)
df.head()

,ETH,BTC,USDT,XRP
2024-11-22 00:00:00,3331.77,99024.03,1.00100,1.4746
2024-11-23 00:00:00,3397.51,97778.77,1.00089,1.4705
2024-11-24 00:00:00,3365.16,98028.18,1.00100,1.4351
2024-11-25 00:00:00,3414.72,93000.01,0.99993,1.4144
2024-11-26 00:00:00,3323.67,91929.54,0.99952,1.3979


In [22]:
df.to_csv("Major_crypto_closing_price.csv")